# Small Research Questions

* I sometimes have questions about how things work that require a quick test.
* I'll try to collect such small test and the lessons here in a compact way.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
from pathlib import Path
current_dir = Path().resolve()
sys.path.append(str(current_dir.parent))
import bibleAssistant.agent

## Response format and JSON

* When calling an LLM with response_format, asking for a valid JSON string, does the schema also force the newline and spaces structure? If so, this may force the LLM to generate a string in different format than the one I used in my training examples, adding confusion.
* Let's do a few tests. Keep in mind this may also be different for Gemma vs. other base LLMs or for ollama's interface vs. openai or other libraries.

In [17]:
import ollama
import datetime

def try_prompt(prompt, force_schema=None):
    print("="*60)
    messages = [{'role':'user', 'content': prompt}]
    print(f"Prompt:\n{prompt}")
    print("-"*20)
    if not force_schema:
        print("(No response format)")
    else:
        print("Forcing response format:")
        print(force_schema)
    print("-"*20)
    resp = ollama.chat(model='gemma3:1b', think=False, messages=messages, format=force_schema).message['content']
    print(resp)

dt = datetime.datetime.now()

prompt1 = f"""
Here is the time and date right now: {dt.strftime("%Y-%m-%d (%A) %H:%M")}.
What is the week day? And what is the year?
Please respond with a valid json string. The JSON object must have field: 
    - 'weekday' with  the name of the weekday.
    - 'year' with the year.
"""
schema1 = {
    'type': 'object',
    'properties':{
        'weekday':{'type':'string'},
        'year': {'type': 'integer'}
    },
    'required':['weekday', 'year']
}

try_prompt(prompt1)
try_prompt(prompt1, schema1)
prompt2 = prompt1+"\nThe JSON string MUST be formatted as a single line without any extra spaces for indentation and without any line breaks!"
try_prompt(prompt2)
try_prompt(prompt2, schema1)
prompt3 = prompt1+"\nThe JSON string MUST be formatted with multi lines and 2-space indentation!"
try_prompt(prompt3)
try_prompt(prompt3, schema1)
prompt4 = prompt1+"\nThe JSON string MUST be formatted as a pretty-print with multi lines and indentation of two spaces!"
try_prompt(prompt4)
try_prompt(prompt4, schema1)
prompt5 = prompt1 + """\nReturn the JSON object with each key and value on its own line.
Indent each nested level by exactly two spaces.
Do not place the entire JSON on one line."""
try_prompt(prompt5)
try_prompt(prompt5, schema1)


Prompt:

Here is the time and date right now: 2026-02-27 (Friday) 11:10.
What is the week day? And what is the year?
Please respond with a valid json string. The JSON object must have field: 
    - 'weekday' with  the name of the weekday.
    - 'year' with the year.

--------------------
(No response format)
--------------------
```json
{
  "weekday": "Friday",
  "year": 2026
}
```

Prompt:

Here is the time and date right now: 2026-02-27 (Friday) 11:10.
What is the week day? And what is the year?
Please respond with a valid json string. The JSON object must have field: 
    - 'weekday' with  the name of the weekday.
    - 'year' with the year.

--------------------
Forcing response format:
{'type': 'object', 'properties': {'weekday': {'type': 'string'}, 'year': {'type': 'integer'}}, 'required': ['weekday', 'year']}
--------------------
{"weekday": "Friday", "year": 2026}

Prompt:

Here is the time and date right now: 2026-02-27 (Friday) 11:10.
What is the week day? And what is the yea

### Conclusions:
* Inconclusive (lol).
* I was hoping to see conclusively that forced json schema (at least for ollama with gemma3-1b) does not force the JSON style. I was hoping to see this by getting different responses for different prompt styling-instructions (with the same forced json schema).
* I didn't see what I wanted. Instead - all the attempts with response format generated the same json style - minified (single line).

In [5]:
resp_no_schema

ChatResponse(model='gemma3:1b', created_at='2026-02-27T14:40:06.3798781Z', done=True, done_reason='load', total_duration=None, load_duration=None, prompt_eval_count=None, prompt_eval_duration=None, eval_count=None, eval_duration=None, message=Message(role='assistant', content='', thinking=None, images=None, tool_name=None, tool_calls=None), logprobs=None)